In [33]:
import requests
import simplejson as json
import pandas as pd
import numpy as np
import os
import json
import math
from openpyxl import load_workbook

In [34]:
notebook_path = os.path.abspath("OLS matching.ipynb")

# Path to config file
config_path = os.path.join(os.path.dirname(notebook_path), "Data/config.json")

# Path to asctb formatted azimuth data
az_path = os.path.join(os.path.dirname(notebook_path), "Data/asctb_formatted_azimuth_data/")

In [35]:
with open(config_path) as config_file:
    config= json.load(config_file)
config

{'asctb_sid': '1tK916JyG5ZSXW_cXfsyZnzXfjyoN-8B2GXLbYD6_vF0',
 'references': [{'name': 'lung',
   'url': 'https://hubmapconsortium.github.io/asctb-azimuth-data-comparison/lung.csv',
   'organ_name': 'lung',
   'asctb_sheet_name': 'Lung_v1.1'},
  {'name': 'pancreas',
   'url': 'https://hubmapconsortium.github.io/asctb-azimuth-data-comparison/pancreas.csv',
   'organ_name': 'pancreas',
   'asctb_sheet_name': 'Pancreas_v1.0'},
  {'name': 'kidney',
   'url': 'https://hubmapconsortium.github.io/asctb-azimuth-data-comparison/kidney.csv',
   'organ_name': 'kidney',
   'asctb_sheet_name': 'Kidney_v1.1'},
  {'name': 'brain',
   'url': 'https://hubmapconsortium.github.io/asctb-azimuth-data-comparison/motor_cortex.csv',
   'organ_name': 'brain',
   'asctb_sheet_name': 'Brain_v1.1'},
  {'name': 'bone_marrow',
   'url': 'https://hubmapconsortium.github.io/asctb-azimuth-data-comparison/bone_marrow.csv',
   'organ_name': 'bone_marrow',
   'asctb_sheet_name': 'Bone_Marrow_v1.1'},
  {'name': 'blood_pmb

In [36]:
# Fetch ASCTB sheet ID from config file
asctb_sheet_id = config["asctb_sid"]

In [37]:
# Fetch Azimuth Data
# AS/2 - Represents Author label
# AS/2/Label -  Represents Ontology label

def fetch_azimuth(az_url,name):
    
    if name+'.csv' in os.listdir(az_path):
        azimuth_df= pd.read_csv (az_path+'/'+name+'.csv',skiprows=10)
    else:
        azimuth_df= pd.read_csv (az_url,skiprows=10)
    
    azimuth_all_cts=[]
    azimuth_all_label=[]
    azimuth_all_label_author=[]
    
    # Filter CT/ID column
    azimuth_ct = azimuth_df.filter(regex=("ID"))
    
    # Filter CT Label column
    azimuth_label = azimuth_df.filter(regex=("AS/[0-9]/LABEL$"))
    
    # Filter Author Label column
    azimuth_label_author = azimuth_df.filter(regex=("AS/[0-9]$"))
    
    # Flatten dataframe to list. Append CT/IDs in all annotation level to a single list and convert it to dataframe.
    for col in azimuth_ct:
        azimuth_all_cts.extend(azimuth_ct[col].tolist())
    azimuth_all_cts=pd.DataFrame(azimuth_all_cts)
    azimuth_all_cts.rename(columns = {0:"CT/ID"},inplace = True)
    
    # Flatten dataframe to list. Append CT Label in all annotation level to a single list and convert it to dataframe.
    for col in azimuth_label:
        azimuth_all_label.extend(azimuth_label[col].tolist())
    azimuth_all_label=pd.DataFrame(azimuth_all_label)
    azimuth_all_label.rename(columns = {0:"CT/LABEL"},inplace = True)
    
    # Flatten dataframe to list. Append Author Label in all annotation level to a single list and convert it to dataframe.
    for col in azimuth_label_author:
        azimuth_all_label_author.extend(azimuth_label_author[col].tolist())
    azimuth_all_label_author=pd.DataFrame(azimuth_all_label_author)
    azimuth_all_label_author.rename(columns = {0:"CT/LABEL.Author"},inplace = True)
    
    # Column bind CT/ID , CT/Label and Author Label column
    azimuth_all_cts_label=pd.concat([azimuth_all_cts,azimuth_all_label,azimuth_all_label_author],axis=1)
    
    # Remove duplicate rows
    azimuth_all_cts_label_unique=azimuth_all_cts_label.drop_duplicates()
    azimuth_all_cts_label_unique.reset_index(drop=True, inplace=True)
    
    # Return flattend dataframe before and after removing duplicates.
    return azimuth_all_cts_label,azimuth_all_cts_label_unique
    

In [38]:
# Fetch Asctb Data
# CT/2 - Represents Author label
# CT/2/Label - Represents Ontology label

def fetch_asctb(sheet_id,asctb_sheet_name):
    
    # Read ASCT+B organ table from google sheet
    asctb_df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={asctb_sheet_name}",skiprows=3) 
    
    # Filter CT/ID column
    asctb_ct = asctb_df.filter(regex=("^CT.*ID$"))
    
    # Filter CT Label column
    asctb_label = asctb_df.filter(regex=("CT/[0-9]/LABEL$"))
    
    # Filter Author Label column
    asctb_label_author = asctb_df.filter(regex=("CT/[0-9]$"))
    
    asctb_all_cts=[]
    asctb_all_label=[]
    asctb_all_label_author=[]

    # Flatten dataframe to list. Append CT/IDs in all annotation level to a single list and convert it to dataframe.
    for col in asctb_ct:
        asctb_all_cts.extend(asctb_ct[col].tolist())
    asctb_all_cts=pd.DataFrame(asctb_all_cts)
    asctb_all_cts.rename(columns = {0:"CT/ID"},inplace = True)

    # Flatten dataframe to list. Append CT Labels in all annotation level to a single list and convert it to dataframe.
    for col in asctb_label:
        asctb_all_label.extend(asctb_label[col].tolist())
    asctb_all_label=pd.DataFrame(asctb_all_label)
    asctb_all_label.rename(columns = {0:"CT/LABEL"},inplace = True)
    
    # Flatten dataframe to list. Append Author Label in all annotation level to a single list and convert it to dataframe.
    for col in asctb_label_author:
        asctb_all_label_author.extend(asctb_label_author[col].tolist())
    asctb_all_label_author=pd.DataFrame(asctb_all_label_author)
    asctb_all_label_author.rename(columns = {0:"CT/LABEL.Author"},inplace = True)
    
    # Column bind CT/ID , CT/Label and Author Label column
    asctb_all_cts_label=pd.concat([asctb_all_cts,asctb_all_label,asctb_all_label_author],axis=1)
    
    # Remove duplicate rows
    asctb_all_cts_label_unique=asctb_all_cts_label.drop_duplicates()
    asctb_all_cts_label_unique.reset_index(drop=True, inplace=True)
    
    # Return flattend dataframe before and after removing duplicates.
    return asctb_all_cts_label,asctb_all_cts_label_unique

In [52]:
# Check whether the Azimuth CT ID (cl_az) is present in ASCT+B. asctb_all_cts_label_unique is a dataframe that contains
# unique CT/ID, Label and author label for a reference organ.

# i and j are the index pointing to corresponding row in Azimuth and ASCT+B dataframe respectively. 
# i is the row number of cl_az in Azimuth dataframe.
# j points to the row number in ASCT+B where cl_az matches in ASCT+B.
# az_row_all,asctb_row_all are global lists used to store these the row number of Azimuth, ASCT+B 

# not_matching_all is a list storing index of Azimuth CT(cl_az) that does not match to any CT in ASCT+B. 
# And if there is a match then we append azimuth row to the list az_row_all and corresponding ASCT+B row number 
# to asctb_row_all


def check_in_asctb(cl_az,i,asctb_all_cts_label_unique,az_row_all,asctb_row_all,not_matching_all):    
    flag=0
    
    for j in range(len(asctb_all_cts_label_unique['CT/ID'])):
        if cl_az == asctb_all_cts_label_unique['CT/ID'][j]:
            az_row_all.append(i)
            asctb_row_all.append(j)
            flag=1
    if flag==0:
        not_matching_all.append(i)

In [41]:
# Check for perfect match between ASCT+B and Azimuth tables and return Azimuth CT mismatches
# For example- If the cell CL:000158 is present in Azimuth and the same cell is present in ASCT+B table for a 
# respective organ then we call it a perfect match i.e CTs that are present in both the data sets.
# Matching is performed based on CT/ID

def perfect_match_for_azimuthct_in_asctb(azimuth_all_cts_label_unique,asctb_all_cts_label_unique):
    
    # az_row_all ,asctb_row_all List to store index number of ASCTB, Azimuth row number where a match is occuring
    # not_matching_all list stores Azimuth row number where CT/ID match is not found
    
    az_row_all=[]
    asctb_row_all=[]
    not_matching_all=[]

    for i in range(len(azimuth_all_cts_label_unique['CT/ID'])):  
        if type(azimuth_all_cts_label_unique['CT/ID'][i])!=np.float64 and type(azimuth_all_cts_label_unique['CT/ID'][i])!=float and azimuth_all_cts_label_unique['CT/ID'][i][:3]=="CL:":
            check_in_asctb(azimuth_all_cts_label_unique['CT/ID'][i],i,asctb_all_cts_label_unique,az_row_all,asctb_row_all,not_matching_all)
        else:
            not_matching_all.append(i)
    
    # Subset Azimuth and ASCTB dataframe by rows were a match is found.
    az_matches_all=azimuth_all_cts_label_unique.loc[az_row_all]
    asctb_matches_all=asctb_all_cts_label_unique.loc[asctb_row_all]

    az_matches_all.reset_index(drop=True,inplace=True)
    asctb_matches_all.reset_index(drop=True,inplace=True)
    
    
    az_matches_all.rename(columns = {"CT/ID":"AZ.CT/ID","CT/LABEL":"AZ.CT/LABEL","CT/LABEL.Author":"AZ.CT/LABEL.Author"},inplace = True)
    asctb_matches_all.rename(columns = {"CT/ID":"ASCTB.CT/ID","CT/LABEL":"ASCTB.CT/LABEL","CT/LABEL.Author":"ASCTB.CT/LABEL.Author"},inplace = True)

    # Cbind both dataframes to show the perfect matches found in one dataframe
    
    perfect_matches_all=pd.concat([az_matches_all,asctb_matches_all],axis=1)
    perfect_matches_all=perfect_matches_all.drop_duplicates()
    perfect_matches_all.reset_index(drop=True, inplace=True)
    
    az_mismatches_all=azimuth_all_cts_label_unique.loc[not_matching_all]
    az_mismatches_all=az_mismatches_all.drop_duplicates()
    az_mismatches_all.reset_index(drop=True, inplace=True)
    
    # retrun Perfect matches and azimuth mismatches
    return perfect_matches_all,az_mismatches_all

In [40]:
# Check whether the ASCTB CT ID (cl_asctb) is present in Azimuth. az_all_cts_label_unique is a dataframe that contains
# unique CT/ID, Label and author label for a reference organ.

# i and j are the index(row number) pointing to corresponding row in ASCT+B and Azimuth dataframe respectively. 
# i is the row number of cl_az in Azimuth dataframe.
# j points to the row number in azimuth where cl_asctb matches in Azimuth.
# az_row_all,asctb_row_all are global lists used to store these the row number of Azimuth, ASCT+B 

# not_matching_all is a list storing index of Asctb CT(cl_asctb) that does not match to any CT in Azimuth. 
# And if there is a match then we append Asctb row to the list asctb_row_all and corresponding Azimuth row number 
# to az_row_all
def check_in_az(cl_asctb,i,az_kidney_all_cts_label_unique,az_row,asctb_row,not_matching):    
    flag=0
    for j in range(len(az_kidney_all_cts_label_unique['CT/ID'])):
        if cl_asctb == az_kidney_all_cts_label_unique['CT/ID'][j]:
            az_row.append(j)
            asctb_row.append(i)
            flag=1
            break
    if flag==0:
        not_matching.append(i)

In [42]:
# Check for mismatches between ASCT+B and Azimuth tables and return Asctb CT mismatches
def perfect_match_for_asctbct_in_azimuth(azimuth_all_cts_label_unique,asctb_kidney_all_cts_label_unique):
    az_row=[]
    asctb_row=[]
    not_matching=[]

    for i in range(len(asctb_kidney_all_cts_label_unique['CT/ID'])):
        if type(asctb_kidney_all_cts_label_unique['CT/ID'][i])!=np.float64 and type(asctb_kidney_all_cts_label_unique['CT/ID'][i])!=float and asctb_kidney_all_cts_label_unique['CT/ID'][i][:3]=="CL:":
            check_in_az(asctb_kidney_all_cts_label_unique['CT/ID'][i],i,azimuth_all_cts_label_unique,az_row,asctb_row,not_matching)
        else:
            not_matching.append(i)

    az_matches=azimuth_all_cts_label_unique.loc[az_row]
    asctb_matches=asctb_kidney_all_cts_label_unique.loc[asctb_row]

    az_matches.reset_index(drop=True,inplace=True)
    asctb_matches.reset_index(drop=True,inplace=True)

    az_matches.rename(columns = {"CT/ID":"AZ.CT/ID","CT/LABEL":"AZ.CT/LABEL","CT/LABEL.Author":"AZ.CT/LABEL.Author"},inplace = True)
    asctb_matches.rename(columns = {"CT/ID":"ASCTB.CT/ID","CT/LABEL":"ASCTB.CT/LABEL","CT/LABEL.Author":"ASCTB.CT/LABEL.Author"},inplace = True)

    perfect_matches=pd.concat([asctb_matches,az_matches],axis=1)

    asctb_mismatches=asctb_kidney_all_cts_label_unique.loc[not_matching]
    asctb_mismatches.reset_index(drop=True,inplace=True)
    
    return asctb_mismatches
    

In [53]:
# Filter out CTs that are not present in Ontology
def incorrect_cts_ebi(mismatches):
    found_in_ols=[]
    not_found_in_ols=[]
    already_checked={}
    
    for i in range(len(mismatches['CT/ID'])):    
        
        if type(mismatches['CT/ID'][i])!=np.float64 and type(mismatches['CT/ID'][i])!=float:
            cl_az=mismatches['CT/ID'][i].replace(":","_")
            if cl_az in already_checked:
                if already_checked[cl_az]=='Y':
                    print("Already checked for mismatch",cl_az)
                    found_in_ols.append(i)
                else:
                    print("Already checked for mismatch",cl_az)
                    not_found_in_ols.append(i)  
                continue            
                
            print("Checking for mismatch",cl_az)
            url = "http://www.ebi.ac.uk/ols/api/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F"
            payload={}
            headers = {
                  'Accept': 'application/json'
                }
            response = requests.request("GET", url+cl_az, headers=headers, data=payload)

            if response.status_code!=200:
                not_found_in_ols.append(i)
                already_checked[cl_az] = 'N'
            else:
                found_in_ols.append(i)
                already_checked[cl_az] = 'Y'
        else:
            not_found_in_ols.append(i)
    
    az_not_found_in_ols=mismatches.loc[not_found_in_ols]
    az_not_found_in_ols.reset_index(drop=True,inplace=True)
    
    az_mismatch_asctb_all = mismatches.loc[found_in_ols]
    az_mismatch_asctb_all.reset_index(drop=True,inplace=True)

    return az_not_found_in_ols,az_mismatch_asctb_all

In [43]:
# Concat hierarchy of traversed CTs in a string and add it to the dataframe
# Example - CL:0000084 (T cell) >> CL:0000542 (lymphocyte) >> CL:0000842 (mononuclear cell) >> CL:0000738 (leukocyte)

def add_hier(azimuth_matches_tree,hierarchy_list_all): 
    found_match=[]
    hier=[]
    len_hier=[]
    asctb_ct=[]
    asctb_label=[]
    for i in range(len(hierarchy_list_all)):
        if len(hierarchy_list_all[i])==3:
            found_match.append("Yes")
            asctb_ct.append(list(hierarchy_list_all[i][0][0].items())[-1][0])
            asctb_label.append(list(hierarchy_list_all[i][0][0].items())[-1][1])
            
        else:
            found_match.append("No")
            asctb_ct.append("Not found")
            asctb_label.append("Not found")
        len_hier.append((len(hierarchy_list_all[i][0][0])))
        x=[]
        for k,v in hierarchy_list_all[i][0][0].items():
            abc=str(k + " (" + v + ")")
            x.append(abc)
        hier.append(x)
    hier_1=[]
    for item in hier:
        hier_1.append(str(" >> ".join(item)))
    hier_1=pd.DataFrame(hier_1,columns=["Hierarchy"])
    found_match=pd.DataFrame(found_match,columns=["Match Found"])
    len_hier=pd.DataFrame(len_hier,columns=["Hierarchy Length"])
    asctb_ct = pd.DataFrame(asctb_ct,columns=["ASCTB.CT/ID"])
    asctb_label = pd.DataFrame(asctb_label,columns=["ASCTB.CT/LABEL"])
    
    azimuth_matches_tree.rename(columns = {"CT/ID":"AZ.CT/ID","CT/LABEL":"AZ.CT/LABEL","CT/LABEL.Author":"AZ.CT/LABEL.Author"},inplace = True)
    df_hier=pd.concat([azimuth_matches_tree,found_match,asctb_ct,asctb_label,len_hier,hier_1],axis=1)
    return df_hier
    

In [44]:
# Concat hierarchy of traversed CTs in a string and add it to the dataframe
# Example - CL:0000084 (T cell) >> CL:0000542 (lymphocyte) >> CL:0000842 (mononuclear cell) >> CL:0000738 (leukocyte)

def add_hier_1(azimuth_matches_tree,hierarchy_list_all): 
    found_match=[]
    hier=[]
    len_hier=[]
    asctb_ct=[]
    asctb_label=[]
    for i in range(len(hierarchy_list_all)):
        if len(hierarchy_list_all[i])==3:
            found_match.append("Yes")
            asctb_ct.append(list(hierarchy_list_all[i][0][0].items())[-1][0])
            asctb_label.append(list(hierarchy_list_all[i][0][0].items())[-1][1])
            
        else:
            found_match.append("No")
            asctb_ct.append("Not found")
            asctb_label.append("Not found")
        len_hier.append((len(hierarchy_list_all[i][0][0])))
        x=[]
        for k,v in hierarchy_list_all[i][0][0].items():
            abc=str(k + " (" + v + ")")
            x.append(abc)
        hier.append(x)
    hier_1=[]
    for item in hier:
        hier_1.append(str(" >> ".join(item)))
    hier_1=pd.DataFrame(hier_1,columns=["Hierarchy"])
    found_match=pd.DataFrame(found_match,columns=["Match Found"])
    len_hier=pd.DataFrame(len_hier,columns=["Hierarchy Length"])
    asctb_ct = pd.DataFrame(asctb_ct,columns=["AZ.CT/ID"])
    asctb_label = pd.DataFrame(asctb_label,columns=["AZ.CT/LABEL"])
    
    azimuth_matches_tree.rename(columns = {"CT/ID":"ASCTB.CT/ID","CT/LABEL":"ASCTB.CT/LABEL","CT/LABEL.Author":"ASCTB.CT/LABEL.Author"},inplace = True)
    df_hier=pd.concat([azimuth_matches_tree,found_match,asctb_ct,asctb_label,len_hier,hier_1],axis=1)
    return df_hier
    

In [48]:
# Check for ASCTB CT in Azimuth

def check_in_az_1(az_kidney_all_cts_label_unique,cl_asctb,i,all_links_asctb,hierarchy):    
    flag=0
    for j in range(len(az_kidney_all_cts_label_unique['CT/ID'])):
        if cl_asctb == az_kidney_all_cts_label_unique['CT/ID'][j]:
            tree_match_asctb_1.append(i)
            tree_match_az_1.append(j)
            flag=1
            hierarchy_list_1.append([[hierarchy],[i],[j]])
            print(cl_asctb,az_kidney_all_cts_label_unique['CT/ID'][j],"Match found")
            break
            
    if flag==0:
        print(cl_asctb)
        ols_call(az_kidney_all_cts_label_unique,cl_asctb,i,all_links_asctb,hierarchy)

In [45]:
# Check for Azimuth CT in ASCTB

def check_in_asctb_1(asctb_all_cts_label_unique,cl_az,i,all_links_az,hierarchy_all):    
    flag=0
    for j in range(len(asctb_all_cts_label_unique['CT/ID'])):
        if cl_az == asctb_all_cts_label_unique['CT/ID'][j]:
            tree_match_asctb_all.append(j)
            tree_match_az_all.append(i)
            flag=1
            hierarchy_list_all.append([[hierarchy_all],[i],[j]])
            print(cl_az,asctb_all_cts_label_unique['CT/ID'][j],"Match found")
            
    if flag==0:
        print(cl_az)
        ols_call_1(asctb_all_cts_label_unique,cl_az,i,all_links_az,hierarchy_all)

In [46]:
# Recursively look for parent CTs of Azimuth/ASCTB CT until a CT match in found ASCTB/ Azimuth. 

def ols_call_1(asctb_all_cts_label_unique,cl_az,i,all_links_az,hierarchy_all):
    url = "http://www.ebi.ac.uk/ols/api/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F"
    payload={}
    headers = {
      'Accept': 'application/json'
    }
    #Azimuth
    try:
        response = requests.request("GET", all_links_az['parents']['href'], headers=headers, data=payload)
    except:
        print("No parent")
        tree_not_match_all.append(i)
        hierarchy_list_all.append([[hierarchy_all],[i]])
        return

    if response.status_code!=200:
        print("Status !=200")
        tree_not_match_all.append(i)
        hierarchy_list_all.append([[hierarchy_all],[i]])

    else:
        result_az= json.loads(response.text)
        all_links_az=result_az['_embedded']['terms'][0]['_links']
        ct_id_az=result_az['_embedded']['terms'][0]['obo_id']
        label_az=result_az['_embedded']['terms'][0]['label']
        hierarchy_all[ct_id_az]=label_az
        if ct_id_az[:-8:-1]=='0000000':
            hierarchy_all[ct_id_az]= label_az
            tree_not_match_all.append(i)
            hierarchy_list_all.append([[hierarchy_all],[i]])
            print(ct_id_az, "No match")
        else:
            hierarchy_all[ct_id_az]= label_az

            check_in_asctb_1(asctb_all_cts_label_unique,ct_id_az,i,all_links_az,hierarchy_all)

In [49]:
# Recursively look for parent CTs of Azimuth/ASCTB CT until a CT match in found ASCTB/ Azimuth.

def ols_call(azimuth_all_cts_label_unique,cl_asctb,i,all_links_asctb,hierarchy):
    url = "http://www.ebi.ac.uk/ols/api/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F"
    payload={}
    headers = {
      'Accept': 'application/json'
    }
    #ASCTB
    try:
        response = requests.request("GET", all_links_asctb['parents']['href'], headers=headers, data=payload)
    except:
        print("No parent")
        tree_not_match_1.append(i)
        hierarchy_list_1.append([[hierarchy],[i]])
        return

    if response.status_code!=200:
        print("Status !=200")
        tree_not_match_1.append(i)
        hierarchy_list_1.append([[hierarchy],[i]])

    else:
        result_asctb= json.loads(response.text)
        all_links_asctb=result_asctb['_embedded']['terms'][0]['_links']
        ct_id_asctb=result_asctb['_embedded']['terms'][0]['obo_id']
        label_asctb=result_asctb['_embedded']['terms'][0]['label']
        hierarchy[ct_id_asctb]=label_asctb
        if ct_id_asctb[:-8:-1]=='0000000':
            hierarchy[ct_id_asctb]= label_asctb
            tree_not_match_1.append(i)
            hierarchy_list_1.append([[hierarchy],[i]])
            print(ct_id_asctb, "No match")
        else:
            hierarchy[ct_id_asctb]= label_asctb

            check_in_az_1(azimuth_all_cts_label_unique,ct_id_asctb,i,all_links_asctb,hierarchy)

In [47]:
# Traverse up Azimuth CTs to get a match in ASCTB 

def tree_traversal_azimuth(az_mismatch_asctb_all,asctb_all_cts_label_unique):
    url = "http://www.ebi.ac.uk/ols/api/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F"
    payload={}

    headers = {
      'Accept': 'application/json'
    }
    for i in range(len(az_mismatch_asctb_all['CT/ID'])):
        hierarchy_all={}
        cl_az=az_mismatch_asctb_all['CT/ID'][i]
        print(cl_az,"Original")
        hierarchy_all[cl_az]=az_mismatch_asctb_all['CT/LABEL'][i]
        cl_az=cl_az.replace(":","_")
        response = requests.request("GET", url+cl_az, headers=headers, data=payload)
        if response.status_code!=200:
            tree_not_match_all.append(i)
            hierarchy_list_all.append([hierarchy_all,i])
        else:
            result_az= json.loads(response.text)
            all_links_az=result_az['_embedded']['terms'][0]['_links']
            ols_call_1(asctb_all_cts_label_unique,cl_az,i,all_links_az,hierarchy_all)
        
    az_matches_tree_all=az_mismatch_asctb_all.loc[tree_match_az_all]
    az_matches_tree_all.reset_index(drop=True,inplace=True)

    asctb_matches_tree_all=asctb_all_cts_label_unique.loc[tree_match_asctb_all]
    asctb_matches_tree_all.reset_index(drop=True,inplace=True)

    az_matches_tree_all.rename(columns = {"CT/ID":"AZ.CT/ID","CT/LABEL":"AZ.CT/LABEL","CT/LABEL.Author":"AZ.CT/LABEL.Author"},inplace = True)
    asctb_matches_tree_all.rename(columns = {"CT/ID":"ASCTB.CT/ID","CT/LABEL":"ASCTB.CT/LABEL","CT/LABEL.Author":"ASCTB.CT/LABEL.Author"},inplace = True)

    az_final_matches =pd.concat([az_matches_tree_all,asctb_matches_tree_all],axis=1)  
    
    az_mismatches_final_all=az_mismatch_asctb_all.loc[tree_not_match_all]
    az_mismatches_final_all.reset_index(drop=True,inplace=True)

    az_mismatches_final_all.rename(columns = {"CT/ID":"AZ.CT/ID","CT/LABEL":"AZ.CT/LABEL","CT/LABEL.Author":"AZ.CT/LABEL.Author"},inplace = True)
    
    return az_final_matches,az_mismatches_final_all

In [50]:
# Traverse up ASCTB CT to get a match in Azimuth 

def tree_traversal_asctb(asctb_mismatch_az,azimuth_all_cts_label_unique):
    url = "http://www.ebi.ac.uk/ols/api/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F"
    payload={}
    #hierarchy={}
    headers = {
      'Accept': 'application/json'
    }
    for i in range(len(asctb_mismatch_az['CT/ID'])):
        hierarchy={}
        cl_asctb=asctb_mismatch_az['CT/ID'][i]
        print(cl_asctb,"Original")
        hierarchy[cl_asctb]=asctb_mismatch_az['CT/LABEL'][i]
        cl_asctb=cl_asctb.replace(":","_")
        response = requests.request("GET", url+cl_asctb, headers=headers, data=payload)
        if response.status_code!=200:
            tree_not_match_1.append(i)
            hierarchy_list_1.append([hierarchy,i])
        else:
            result_asctb= json.loads(response.text)
            all_links_asctb=result_asctb['_embedded']['terms'][0]['_links']
            ols_call(azimuth_all_cts_label_unique,cl_asctb,i,all_links_asctb,hierarchy) 

    asctb_matches_tree=asctb_mismatch_az.loc[tree_match_asctb_1]
    asctb_matches_tree.reset_index(drop=True,inplace=True)

    az_matches_tree=azimuth_all_cts_label_unique.loc[tree_match_az_1]
    az_matches_tree.reset_index(drop=True,inplace=True)

    az_matches_tree.rename(columns = {"CT/ID":"AZ.CT/ID","CT/LABEL":"AZ.CT/LABEL","CT/LABEL.Author":"AZ.CT/LABEL.Author"},inplace = True)
    asctb_matches_tree.rename(columns = {"CT/ID":"ASCTB.CT/ID","CT/LABEL":"ASCTB.CT/LABEL","CT/LABEL.Author":"ASCTB.CT/LABEL.Author"},inplace = True)

    az_final_matches =pd.concat([asctb_matches_tree,az_matches_tree],axis=1)


    asctb_mismatches_final=asctb_mismatch_az.loc[tree_not_match_1]
    asctb_mismatches_final.reset_index(drop=True,inplace=True)

    asctb_mismatches_final.rename(columns = {"CT/ID":"ASCTB.CT/ID","CT/LABEL":"ASCTB.CT/LABEL","CT/LABEL.Author":"ASCTB.CT/LABEL.Author"},inplace = True)

    return az_final_matches,asctb_mismatches_final

In [51]:
# Set differene between dataframes
def uq_ct_df(incorrect_ct_ebi,cts_uq):
    set_diff_df = pd.concat([cts_uq, incorrect_ct_ebi, incorrect_ct_ebi]).drop_duplicates(keep=False)
    return(set_diff_df)

In [54]:
# Generate summary for all organs

summary = pd.DataFrame(columns=["Organ","Az_missing_cts","Asctb_missing_cts","Az_Asctb_perfect_matches","Az_unique_CT","ASCTB_unique_CT","Az_cts_not_matched","Az_pecentage_not_matched","Asctb_cts_not_matched","Asctb_percentage_not_matched","Az_incorrect_cts","Asctb_incorrect_cts","Az_ct_match_found_crosswalk","Asctb_ct_match_found_crosswalk"])
ct=0
loc=0

# Loop over all organ in reference file

for ref in config['references']:
    name= ref['name']
    asctb_sheet_name = ref['asctb_sheet_name']
    az_url= ref['url']
    

    # Fetch Azimuth data
    azimuth_all_cts_label,azimuth_all_cts_label_unique = fetch_azimuth(az_url,name)
    azimuth_all_cts_label = azimuth_all_cts_label.dropna(axis = 0, how = 'all', inplace = False)
    azimuth_all_cts_label_unique = azimuth_all_cts_label_unique.dropna(axis = 0, how = 'all', inplace = False)
    azimuth_all_cts_label_unique.reset_index(drop=True, inplace=True)
    
    # Fetch ASCTB data
    asctb_all_cts_label,asctb_all_cts_label_unique = fetch_asctb(asctb_sheet_id,asctb_sheet_name)
    asctb_all_cts_label = asctb_all_cts_label.dropna(axis = 0, how = 'all', inplace = False)
    asctb_all_cts_label_unique = asctb_all_cts_label_unique.dropna(axis = 0, how = 'all', inplace = False)
    asctb_all_cts_label_unique.reset_index(drop=True, inplace=True)
    
    # Number of Azimuth cts without IDs
    azimuth_missing_cts = azimuth_all_cts_label_unique[azimuth_all_cts_label_unique['CT/ID'].isna() & ~azimuth_all_cts_label_unique['CT/LABEL'].isna()].reset_index(drop=True)
    
    # Number of ASCTB cts without IDs
    asctb_missing_cts = asctb_all_cts_label_unique[asctb_all_cts_label_unique['CT/ID'].isna() & ~asctb_all_cts_label_unique['CT/LABEL'].isna()].reset_index(drop=True)
    
    # Perfect Match and Mismatch for Azimuth CT in ASCTB (AZ - ASCTB)
    azimuth_perfect_matches,azimuth_mismatches=perfect_match_for_azimuthct_in_asctb(azimuth_all_cts_label_unique,asctb_all_cts_label_unique)
    azimuth_perfect_matches.sort_values(by=['AZ.CT/ID','AZ.CT/LABEL.Author'],inplace=True)

    # Mismatch for ASCTB CT in Azimuth (ASCTB - Azimuth)
    asctb_mismatches=perfect_match_for_asctbct_in_azimuth(azimuth_all_cts_label_unique,asctb_all_cts_label_unique)
    
    # Remove rows having missing CT/IDs
    azimuth_mismatches = azimuth_mismatches[~azimuth_mismatches['CT/ID'].isna()].reset_index(drop=True)
    asctb_mismatches = asctb_mismatches[~asctb_mismatches['CT/ID'].isna()].reset_index(drop=True)
    
    # Incorrect CT ID in Azimuth (EBI)
    incorrect_ct_azimuth_ebi, azimuth_mismatches_filtered=incorrect_cts_ebi(azimuth_mismatches)
    incorrect_ct_azimuth_ebi_ct = incorrect_ct_azimuth_ebi.iloc[:,0].drop_duplicates()
    incorrect_ct_azimuth_ebi_ct.reset_index(drop=True, inplace=True)
    incorrect_ct_azimuth_ebi_ct.dropna(axis = 0, how = 'all', inplace = True)
    incorrect_ct_azimuth_ebi_ct.reset_index(drop=True, inplace=True)
        
    # Incorrect CT ID in Asctb (EBI)
    incorrect_ct_asctb_ebi, asctb_mismatches_filtered=incorrect_cts_ebi(asctb_mismatches)
    incorrect_ct_asctb_ebi_ct = incorrect_ct_asctb_ebi.iloc[:,0].drop_duplicates()
    incorrect_ct_asctb_ebi_ct.reset_index(drop=True, inplace=True)
    incorrect_ct_asctb_ebi_ct.dropna(axis = 0, how = 'all', inplace = True)
    incorrect_ct_asctb_ebi_ct.reset_index(drop=True, inplace=True)
    
    
    asctb_cts_uq=asctb_all_cts_label_unique.iloc[:,0].drop_duplicates()
    asctb_cts_uq.reset_index(drop=True, inplace=True)
    
    azimuth_cts_uq=azimuth_all_cts_label_unique.iloc[:,0].drop_duplicates()
    azimuth_cts_uq.reset_index(drop=True, inplace=True)
    
    azimuth_cts_uq.dropna(axis = 0, how = 'all', inplace = True)
    asctb_cts_uq.dropna(axis = 0, how = 'all', inplace = True)
    
    ct_asctb = uq_ct_df(incorrect_ct_asctb_ebi_ct,asctb_cts_uq)
    ct_azimuth = uq_ct_df(incorrect_ct_azimuth_ebi_ct,azimuth_cts_uq)
    
    # Tree traversal for matching Az to Asctb. Traversing up Azmiuth
    tree_match_asctb_all=[]
    tree_match_az_all=[]
    tree_not_match_all=[]
    hierarchy_list_all=[]
    
    azimuth_matches_tree,azimuth_mismatches_tree  = tree_traversal_azimuth(azimuth_mismatches_filtered,asctb_all_cts_label_unique)
    azimuth_matches_tree_hier = add_hier(azimuth_mismatches_filtered,hierarchy_list_all)
        
    # Tree traversal for matching Asctb to Az. Traversing up Asctb
    tree_match_asctb_1=[]
    tree_match_az_1=[]
    tree_not_match_1=[]
    hierarchy_list_1=[]
    
    asctb_matches_tree_all,asctb_mismatch_tree = tree_traversal_asctb(asctb_mismatches_filtered,azimuth_all_cts_label_unique)
    asctb_matches_tree_hier = add_hier_1(asctb_mismatches_filtered,hierarchy_list_1)

    set_diff_df = pd.concat([azimuth_mismatches_tree['AZ.CT/ID'], asctb_matches_tree_all['AZ.CT/ID'].drop_duplicates().reset_index(drop=True), asctb_matches_tree_all['AZ.CT/ID'].drop_duplicates().reset_index(drop=True)]).drop_duplicates(keep=False)
    idx=set_diff_df.index.tolist()
    azimuth_mismatches_tree= azimuth_mismatches_tree.filter(items=idx,axis=0)
    
    
    set_diff_df = pd.concat([asctb_mismatch_tree['ASCTB.CT/ID'], azimuth_matches_tree['ASCTB.CT/ID'].drop_duplicates().reset_index(drop=True), azimuth_matches_tree['ASCTB.CT/ID'].drop_duplicates().reset_index(drop=True)]).drop_duplicates(keep=False)
    idx=set_diff_df.index.tolist()
    asctb_mismatch_tree= asctb_mismatch_tree.filter(items=idx,axis=0)
    
    
    # Drop all na 
    azimuth_missing_cts.dropna(axis = 0, how = 'all', inplace = True)
    asctb_missing_cts.dropna(axis = 0, how = 'all', inplace = True)
    azimuth_perfect_matches.dropna(axis = 0, how = 'all', inplace = True)
    incorrect_ct_azimuth_ebi.dropna(axis = 0, how = 'all', inplace = True)
    incorrect_ct_asctb_ebi.dropna(axis = 0, how = 'all', inplace = True)
    azimuth_matches_tree_hier.dropna(axis = 0, how = 'all', inplace = True)
    asctb_matches_tree_hier.dropna(axis = 0, how = 'all', inplace = True)
    asctb_mismatch_tree.dropna(axis = 0, how = 'all', inplace = True)
    azimuth_mismatches_tree.dropna(axis = 0, how = 'all', inplace = True)
    
    azimuth_matches_tree_hier_y= azimuth_matches_tree_hier[azimuth_matches_tree_hier["Match Found"]=="Yes"]
    asctb_matches_tree_hier_y= asctb_matches_tree_hier[asctb_matches_tree_hier["Match Found"]=="Yes"]
    
    azimuth_matches_tree_hier_n= azimuth_matches_tree_hier[azimuth_matches_tree_hier["Match Found"]=="No"]
    asctb_matches_tree_hier_n= asctb_matches_tree_hier[asctb_matches_tree_hier["Match Found"]=="No"]
    
    azimuth_matches_tree_hier_len = sum(azimuth_matches_tree_hier["Match Found"]=="Yes")
    asctb_matches_tree_hier_len = sum(asctb_matches_tree_hier["Match Found"]=="Yes")
    
    # Create a list of final matches i.e perfect match and crosswalk match
    a = azimuth_perfect_matches[['AZ.CT/LABEL','ASCTB.CT/LABEL']].drop_duplicates().reset_index(drop=True)
    b = azimuth_matches_tree_hier_y[['AZ.CT/LABEL','ASCTB.CT/LABEL']].drop_duplicates().reset_index(drop=True)
    c = asctb_matches_tree_hier_y[['AZ.CT/LABEL','ASCTB.CT/LABEL']].drop_duplicates().reset_index(drop=True)
    final_match =  pd.concat([a,b,c])
    final_match.drop_duplicates().reset_index(drop=True,inplace=True)
    
    
    print("---------------------------------------------------------------------------------------------------")
    print(name)
    #print("ASCTB Mismatches",len(asctb_mismatches))
    #print("Azimuth Mismatches",len(azimuth_mismatches))
    print("Unique Azimuth Total CT&Label",len(azimuth_all_cts_label_unique))
    print("Unique ASCTB Total CT&Label",len(asctb_all_cts_label_unique))
    print("Perfect Matches",len(azimuth_perfect_matches))
    print("Ct Matches (Traversing up Azmiuth)",len(azimuth_matches_tree_hier_y))
    print("Ct Matches (Traversing up Asctb)",len(asctb_matches_tree_hier_y))
    print("Final mismatch (Azimuth)",len(azimuth_mismatches_tree))
    print("Final mismatch (ASCTB)",len(asctb_mismatch_tree))
    
    
    # Calculate percentage missing
    az_percent_not_matching = len(azimuth_mismatches_tree)*100/(len(azimuth_all_cts_label_unique) if len(ct_azimuth)!=0 else 1)
    asctb_percent_not_matching = len(asctb_mismatch_tree)*100/(len(asctb_all_cts_label_unique) if len(ct_asctb)!=0 else 1)
    
    print("Percentage not matching (Azimuth)",az_percent_not_matching)
    print("Percentage not matching (Asctb)",asctb_percent_not_matching)
    
    print("---------------------------------------------------------------------------------------------------")
    
    with pd.ExcelWriter("./Data/Final/"+name+ ".xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
        azimuth_missing_cts.to_excel(writer, sheet_name="Az_missing_cts", index=False)
        asctb_missing_cts.to_excel(writer, sheet_name="Asctb_missing_cts", index=False)
        
        azimuth_perfect_matches.to_excel(writer, sheet_name="Az_Asctb_cts_perfect_matches", index=False)
        
        incorrect_ct_azimuth_ebi.to_excel(writer, sheet_name="Az_incorrect_cts", index=False)
        incorrect_ct_asctb_ebi.to_excel(writer, sheet_name="Asctb_incorrect_cts", index=False)
        
        azimuth_matches_tree_hier_y = azimuth_matches_tree_hier_y.sort_values(by=['AZ.CT/ID','AZ.CT/LABEL.Author'],inplace=False)
        asctb_matches_tree_hier_y = asctb_matches_tree_hier_y.sort_values(by=['ASCTB.CT/ID','ASCTB.CT/LABEL.Author'],inplace=False)
        
        azimuth_matches_tree_hier_y.to_excel(writer, sheet_name="Az_match_tree_crosswalk", index=False)
        asctb_matches_tree_hier_y.to_excel(writer, sheet_name="Asctb_match_tree_crosswalk", index=False)
        
        azimuth_mismatches_tree = azimuth_mismatches_tree.sort_values(by=['AZ.CT/ID','AZ.CT/LABEL.Author'],inplace=False)
        asctb_mismatch_tree = asctb_mismatch_tree.sort_values(by=['ASCTB.CT/ID','ASCTB.CT/LABEL.Author'],inplace=False)
        
        azimuth_mismatches_tree.to_excel(writer, sheet_name="Az_cts_mismatch_final", index=False)
        asctb_mismatch_tree.to_excel(writer, sheet_name="Asctb_cts_mismatch_final", index=False)
        
        final_match.to_excel(writer, sheet_name="Final_Matches", index=False)

         
    summary.loc[loc] = [name]+ [len(azimuth_missing_cts)]+[len(asctb_missing_cts)]+[len(azimuth_perfect_matches)] + [len(azimuth_all_cts_label_unique)] +[len(asctb_all_cts_label_unique)] + [len(azimuth_mismatches_tree)]+ [az_percent_not_matching]+[len(asctb_mismatch_tree)]+[asctb_percent_not_matching] + [len(incorrect_ct_azimuth_ebi)] + [len(incorrect_ct_asctb_ebi)]+ [azimuth_matches_tree_hier_len] + [asctb_matches_tree_hier_len]
    loc+=1    
    

Checking for mismatch CL_0000057
Checking for mismatch CL_0000192
Checking for mismatch UBERON_0001637
Checking for mismatch CL_0000646
Checking for mismatch CL_0000766
Checking for mismatch CL_0002328
Checking for mismatch UBERON_0001982
Checking for mismatch CL_0002144
Already checked for mismatch CL_0002144
Checking for mismatch CL_0000624
Checking for mismatch CL_0000625
Checking for mismatch CL_0000064
Checking for mismatch CL_0002057
Checking for mismatch CL_0000451
Already checked for mismatch CL_0000192
Checking for mismatch CL_0000160
Checking for mismatch CL_0002396
Checking for mismatch CL_0005006
Already checked for mismatch CL_0000057
Checking for mismatch UBERON_0001473
Checking for mismatch CL_0000235
Checking for mismatch CL_0000077
Checking for mismatch CL_0000319
Checking for mismatch CL_0000165
Checking for mismatch CL_0000763
Already checked for mismatch CL_0000235
Checking for mismatch CL_0000542
Checking for mismatch CL_0000313
Checking for mismatch UBERON_0001638

CL:0000738
CL:0000219
CL:0000003
CL:0000000 No match
CL:0000646 Original
CL:0000036
CL:0000035
CL:0000723
CL:0002371
CL:0000003
CL:0000000 No match
CL:0000646 Original
CL:0000036
CL:0000035
CL:0000723
CL:0002371
CL:0000003
CL:0000000 No match
CL:0000064 Original
CL:0000003
CL:0000000 No match
CL:0000451 Original
CL:0000842
CL:0000738
CL:0000219
CL:0000003
CL:0000000 No match
CL:0000359 Original
CL:0000192
CL:0008000
CL:0000187
CL:0000183
CL:0000003
CL:0000000 No match
CL:1000348 Original
CL:0000307
CL:0002202
CL:0002632
CL:0002368
CL:0002076
CL:0002371
CL:0000003
CL:0000000 No match
CL:1000329 Original
CL:0000307
CL:0002202
CL:0002632
CL:0002368
CL:0002076
CL:0002371
CL:0000003
CL:0000000 No match
CL:0002145 Original
CL:0005012
CL:0000067
CL:0000064 CL:0000064 Match found
CL:1000377 Original
CL:0000307
CL:0002202
CL:0002632
CL:0002368
CL:0002076
CL:0002371
CL:0000003
CL:0000000 No match
CL:1000330 Original
CL:0019001
CL:0000313 CL:0000313 Match found
CL:0002600 Original
CL:0019019
CL:0

Checking for mismatch CL_0002573
Checking for mismatch CL_0000785
Checking for mismatch CL_0000451
Checking for mismatch CL_0005011
Checking for mismatch [kidney loop of Henle cortical thick ascending limb epithelial cell](http_//www.ontobee.org/ontology/CL?iri=http_//purl.obolibrary.org/obo/1001109)
Already checked for mismatch CL_0000359
Checking for mismatch CL_0000113
Checking for mismatch CL_1000547
Checking for mismatch CL_0000235
Already checked for mismatch CL_0000057
Checking for mismatch CL_0000650
Checking for mismatch CL_0000542
Checking for mismatch Outer Medullary Collecting Duct Intercalated Type A
Checking for mismatch Outer Medullary Collecting Duct Principal
Checking for mismatch Papillary Tip Epithelial
Checking for mismatch Parietal Epithelial
Checking for mismatch Peritubular Capilary Endothelial 
Checking for mismatch CL_0000784
Already checked for mismatch CL_0002573
Already checked for mismatch CL_0005011
Checking for mismatch CL_1000742
Checking for mismatch CL

Already checked for mismatch CL_0000128
Already checked for mismatch CL_0000128
Already checked for mismatch CL_0000128
Already checked for mismatch CL_0002453
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617
Already checked for mismatch CL_0000617


CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000679 Original
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:00001

CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No match
CL:0000617 Original
CL:0000161
CL:0000151
CL:0000003
CL:0000000 No matc

Checking for mismatch CL_0000945
Checking for mismatch CL_0001056
Checking for mismatch CL_0000775
Checking for mismatch CL_0000094
Checking for mismatch CL_0000084
Checking for mismatch CL_0000814
Checking for mismatch CL_0000816
Checking for mismatch CL_0000818
Checking for mismatch CL_0001058
Checking for mismatch CL_0000840
Checking for mismatch CL_0001054
Already checked for mismatch CL_0000775
Checking for mismatch CL_0000771
Checking for mismatch CL_0000767
Checking for mismatch CL_0001062
Checking for mismatch CL_0000824
CL:0000451 Original
CL:0000842
CL:0000738
CL:0000219
CL:0000003
CL:0000000 No match
CL:0000785 Original
CL:0001201
CL:0000236
CL:0000945 CL:0000945 Match found
CL:0000624 Original
CL:0000791
CL:0000789
CL:0000084 CL:0000084 Match found
CL:0000625 Original
CL:0000791
CL:0000789
CL:0000084 CL:0000084 Match found
CL:0002419 Original
CL:0000084 CL:0000084 Match found
CL:0000623 Original
CL:0001067
CL:0001065
CL:0000542
CL:0000842
CL:0000738
CL:0000219
CL:0000003
CL

In [55]:
with pd.ExcelWriter("./Data/Final/summary.xlsx") as writer:
    summary.to_excel(writer, sheet_name="Summary", index=False)